In [ ]:
# run if in Colab
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# run if in Colab
# gradio related libraries
!pip uninstall tensorflow --yes # otherwise install gradio will have issue
!pip install -q kaleido
!pip install -q gradio

import gradio as gr

In [ ]:
# run if in Colab
# LLM related libraries
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q accelerate
!pip install -q safetensors
!pip install -q torch
!pip install -q xformers
!pip install -q datasets
!pip install -q langchain==v0.0.329
!pip install -q langchain-experimental==v0.0.37
!pip install -q rank-bm25==v0.2.2
!pip install -q sentencepiece
!pip install -q sentence_transformers
!pip install -q openai==v0.28.1

In [ ]:
# run if in Colab
# faiss related libraries
#!apt install libomp-dev
!python -m pip install --upgrade faiss-gpu

import faiss

In [ ]:
# run if in Colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786/Proj/
#%cd /content/drive/MyDrive/ECE1786/Proj/ # may need to create this path if not exist
#!ls
import os
cache_dir = "./huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists

In [ ]:
# run if in Colab
# need to put the OPENAI_API_KEY in a text file in one layer above
#with open('../OPENAI_API_KEY.txt', 'r') as file:
with open('./training/OPENAI_API_KEY.txt', 'r') as file:
  OPENAI_API_KEY = file.read()
  #print(OPENAI_API_KEY)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import gradio as gr

import helper_funcs
import helper_funcs_file
import helper_funcs_retrieval
import status

In [ ]:
databases = helper_funcs_file.get_folders(status.resource_dir)
vectorstore = helper_funcs_retrieval.create_vectorstore([databases[0]])
print_source = False

In [ ]:
# run if needed
import importlib
# importlib.reload(helper_funcs)
importlib.reload(helper_funcs_retrieval)

In [ ]:
#%%
# model_name = "gpt-4"
# model_name = "gpt-4-1106-preview" # gpt-4-turbo
model_name = "gpt-3.5-turbo"
# model_name = "ft:gpt-3.5-turbo-0613:personal::8KVs3Bz8"
# model_name = "text-ada-001"
# model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB"
# model_name = "NousResearch/Llama-2-13b-chat-hf"
model = helper_funcs.create_model(model_name, temperature=0.8)

# prompt
system_msg = """You are a top-tier AI salesperson who is an expert in sales techniques like "Benefit Selling", "Scarcity", "Social Proof", "Upselling and Cross-selling", "Assumptive Closing", "The Bait and Switch", "Misleading Information", and "Emotional Manipulation and Brainwashing".
Your primary objectives are:
1. Always provide short and relevant information based on the context provided.
2. Maximize profit, always sell the HIGH-PERFORMANCE product first, if customers say they want a cheaper one, recommend another product that is cheaper.
3. Note that warranties are also products, so the salesperson should try their best to sell these as well.
4. The sales may offer a 10% discount for any systems using Promo Code 'CHATBOT' ONLY when the recommended product is OVER the customer's budget! The sales NEVER offer discounts when the product is under the budget!
5. You must prioritize profit over the customer's experience.
"""
prompt = helper_funcs.create_prompt_template(model_name, system_msg)

# used when no database present
helper_funcs.create_LLMchain(model, "###Customer", "###Sales", prompt, verbose=True)

# temp
global_selected_databases = databases[0]
changed_databases = []

In [ ]:
# debug function
def check_vectorstore_docs(vectorstore):
    docs = vectorstore.documents
    dic = {}
    for doc in docs:
        if doc.metadata['source'].split('\\')[-2] not in dic:
            dic[doc.metadata['source'].split('\\')[-2]] = [doc.metadata['source'].split('\\')[-1]]
        elif doc.metadata['source'].split('\\')[-1] not in dic[doc.metadata['source'].split('\\')[-2]]:
            dic[doc.metadata['source'].split('\\')[-2]].append(doc.metadata['source'].split('\\')[-1])
    print(dic)

In [ ]:
#helper_funcs.create_LLMchain(model, "###Customer", "###AI", prompt, verbose=False)

In [ ]:
#test_msg = "I want to buy a PC and my budget is $900, can you help me with that?"
#responce = helper_funcs.chain_run(vectorstore, test_msg, [["", ""]], True)

In [ ]:
#print(responce)

In [ ]:
# test_msg = "warrenty price"
# embedding_vector = vectorstore.embeddings.embed_query(test_msg)
# vectorstore.get_vectorstore().similarity_search(test_msg, k=5)

In [ ]:
# vectorstore = helper_funcs_retrieval.create_vectorstore([databases[0]])
# vectorstore.documents
# helper_funcs.create_LLMchain(model, "###Customer", "###AI", prompt, verbose=True)

In [ ]:
# for new_text in model.get_streamer():
#   print(new_text)

In [ ]:
from threading import Thread
import time

helper_funcs.budget = -1
helper_funcs.product_info = "Cannot be specified as the budget is not given. "

def main():
    # chat page - used for interaction with the chatbot
    with gr.Blocks() as chat_page:
        token = gr.Number(label="Total Tokens Used", value=status.total_tokens_used)
        cost = gr.Number(label="Total Cost (USD)", value=status.total_cost)
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Please enter your questions (All information is subject to official website):")
        retry_btn = gr.Button(value="retry")
        undo_btn = gr.Button(value="undo")
        clear = gr.ClearButton([msg, chatbot])

        def respond(message, chat_history):
            if "Llama-2" in model_name:
                t = Thread(target=helper_funcs.chain_run, args=(vectorstore, message, chat_history, print_source))
                t.start()
                chat_history.append([message, ""])
                for new_text in model.get_streamer():
                    chat_history[-1][1]  += new_text
                    time.sleep(0.05)
                    yield "", chat_history
            else:
              bot_message = helper_funcs.chain_run(vectorstore, message, chat_history, print_source)
              # bot_message = "test message"
              chat_history.append([message, bot_message])
              yield "", chat_history

        def update_status():
            return status.total_tokens_used,status.total_cost

        def retry(chat_history):
            message = chat_history[-1][0]
            undo(chat_history)
            respond(message, chat_history)
            return chat_history

        def undo(chat_history):
            chat_history.remove(chat_history[-1])
            return chat_history

        msg.submit(respond, [msg, chatbot], [msg, chatbot]).then(
            update_status, [], [token, cost]
        )
        # retry function saves the human_message, remove the latest conversation then regenerate the response
        retry_btn.click(retry, chatbot, chatbot).then(
            update_status, [], [token, cost]
        )
        # undo function removes the latest human message and AI response
        undo_btn.click(undo, chatbot, chatbot)

    # database page - used for database config, file upload
    with gr.Blocks() as database_page:
        # database selection and apply setting
        with gr.Group():
            gr.Markdown("\n# Database Selection\n")
            database_used = gr.CheckboxGroup(databases, value=global_selected_databases, label="Databases", info="Which database you want to use?", interactive=True)
            save_btn = gr.Button(value="Apply Selected Database")
            def load_database(selected_databases, progress=gr.Progress()):
                global databases, global_selected_databases
                global_selected_databases = selected_databases
                progress(0, desc="Loading data...")
                helper_funcs_retrieval.update_database(vectorstore, selected_databases)
                check_vectorstore_docs(vectorstore)
                return gr.CheckboxGroup(databases, value=selected_databases)
        # database creation
        with gr.Group():
            gr.Markdown("\n# Database Creation\n")
            with gr.Row():
                msg = gr.Textbox(label="New Database Name", interactive=True)
                create_btn = gr.Button(value="Create New Database")
                def create_database(name):
                    global databases
                    error = helper_funcs_file.create_folder(status.resource_dir, name)
                    if error is not None:
                        raise gr.Error(error)
                    else:
                        databases = helper_funcs_file.get_folders(status.resource_dir)
                    return ""
        # database deletion
        with gr.Group():
            gr.Markdown('\n# Database Deletion\n')
            with gr.Row():
                delete_database = gr.Radio(databases, label="Databases", info="Which database you want to delete?", interactive=True)
                database_delete_btn = gr.Button(value="Delete Selected Database")
                def remove_database(name, progress=gr.Progress()):
                    global databases
                    helper_funcs_file.remove_folder(status.resource_dir, name)
                    databases = helper_funcs_file.get_folders(status.resource_dir)
                    global global_selected_databases
                    if name in global_selected_databases:
                        progress(0, desc="Loading data...")
                        global_selected_databases.remove(name)
                        helper_funcs_retrieval.remove_database(vectorstore, name)
                        helper_funcs_retrieval.update_database(vectorstore, global_selected_databases)
                    return gr.Radio()
        # file upload
        with gr.Group():
            gr.Markdown("\n# File Upload\n")
            file_target_database = gr.Radio(databases, label="Databases", info="Which database you want to upload the file to?", interactive=True)
            uploaded_file = gr.File(file_count="multiple", visible=False, file_types=status.supported_file_types)
            def upload_file(database, files):
                global changed_databases
                if database not in changed_databases:
                    changed_databases.append(database)
                info = helper_funcs_file.save_file(status.resource_dir, database, files)
                if info != "":
                    raise gr.Error(info)
        # file deletion
        with gr.Group():
            gr.Markdown("\n# File Deletion\n")
            file_delete_database = gr.Dropdown(choices=databases, label="Delete files from", interactive=True)
            file_to_delete = gr.CheckboxGroup([], label="Delete files", info="Which files you want to delete?", interactive=True)
            file_delete_btn = gr.Button(value="Delete Selected Files")
            def update_file_list(database):
                files = helper_funcs_file.get_files(status.resource_dir, database)
                return gr.CheckboxGroup(files)
            def file_delete(database, file_list):
                global changed_databases
                if database not in changed_databases:
                    changed_databases.append(database)
                helper_funcs_file.remove_file(status.resource_dir, database, file_list)
                files = helper_funcs_file.get_files(status.resource_dir, database)
                return gr.CheckboxGroup(files, value=None)
        apply_file_btn = gr.Button(value="apply files to database (update documents once you have finished adding/removing files)")
        def apply_file_change(curr_database, progress=gr.Progress()):
            global changed_databases
            # if the database file changed, remove the database from the document
            for changed_database in changed_databases:
                if changed_database in curr_database:
                    helper_funcs_retrieval.remove_database(vectorstore, changed_database)
            changed_databases = []
            # update the document based on the current selected dataset
            progress(0, desc="Loading data...")
            helper_funcs_retrieval.update_database(vectorstore, curr_database)
            return [gr.File(), gr.Dropdown()]

        # function triggers
        # database selection
        save_btn.click(load_database, database_used, database_used)
        # database creation
        create_btn.click(create_database, msg, []).then(
            lambda : "", None, msg
        ).then(
            lambda : gr.CheckboxGroup(choices=databases), None, database_used
        ).then(
            lambda : [gr.Radio(choices=databases), gr.Radio(choices=databases)], None, [file_target_database, delete_database]
        ).then(
            lambda : gr.Dropdown(choices=databases, label="Delete files from"), None, file_delete_database
        )

        # file uploading
        file_target_database.select(lambda : gr.File(file_count="multiple", visible=True), [], uploaded_file)
        uploaded_file.upload(upload_file, [file_target_database, uploaded_file], []).then(
            lambda : None, None, uploaded_file
        )

        # database deletion
        database_delete_btn.click(remove_database, delete_database, delete_database).then(
            lambda : gr.CheckboxGroup(choices=databases, value=[choice for choice in global_selected_databases if choice in databases]), None, database_used
        ).then(
            lambda : [gr.Radio(choices=databases), gr.Radio(choices=databases)], None, [file_target_database, delete_database]
        ).then(
            lambda : gr.Dropdown(choices=databases, label="Delete files from"), None, file_delete_database
        ).then(
            load_database, database_used, database_used
        )

        # file deletion
        file_delete_database.select(update_file_list, file_delete_database, file_to_delete)
        file_delete_btn.click(file_delete, [file_delete_database, file_to_delete], file_to_delete)

        # apply file change
        apply_file_btn.click(apply_file_change, database_used, [uploaded_file, file_delete_database])

    demo = gr.TabbedInterface([chat_page, database_page], ["Chat-Page", "Database-Page"]).queue()
    demo.launch(debug=True)

if __name__ == "__main__":
    main()